<a href="https://www.kaggle.com/code/theongy/getting-started-maneuver-detection?scriptVersionId=120424218" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
"""Command line to load the data (valid only for this notebook)"""
import sys
package_paths = [
    '../input/hackathon2023/',
]

for pth in package_paths:
    sys.path.append(pth)
!cp -r '../input/maneuver-detection/' '/kaggle/working'

# Loading the dataset

A custom pytorch dataset is provided. It will parse the json dataset can be used with the pytorch loader. Since loading can sometimes take time, it is recommanded to execute the following cell at the beginning separately.

In [2]:
from ManeuverDetectionDataset import ManeuverDetectionDataset
dataset_path="/kaggle/working/maneuver-detection/TRAIN_1_IRREGULAR_STEPS_V2.json"
train_dataset= ManeuverDetectionDataset(dataset_path)



**********TRAIN DATASET *********
Validation/Train ratio: 0.1
Samples filtered? NO
Samples evenly spaced? False
path: /kaggle/working/maneuver-detection/TRAIN_1_IRREGULAR_STEPS_V2.json
loading dataset. Ready in a minute!
TRAIN Dataset loaded. Size: 35762


As you can see, by default, the dataset is considered as a train dataset and 10% is not loaded for validation. To build the validation dataset, yo can do the same and you don't need to load again the json to save time ! **Beware : if you modify the code, you should take care to avoid to validate your model with the trained data!**

In [3]:
dataset= ManeuverDetectionDataset(dataset_path,dataset_type="VALIDATION",imported_dataset=train_dataset.dataset)



**********VALIDATION DATASET *********
Validation/Train ratio: 0.1
Samples filtered? NO
Samples evenly spaced? False
path: /kaggle/working/maneuver-detection/TRAIN_1_IRREGULAR_STEPS_V2.json
VALIDATION Dataset loaded. Size: 3974


In [4]:
validation_dataset= ManeuverDetectionDataset(dataset_path,dataset_type="VALIDATION",imported_dataset=train_dataset.dataset)



**********VALIDATION DATASET *********
Validation/Train ratio: 0.1
Samples filtered? NO
Samples evenly spaced? False
path: /kaggle/working/maneuver-detection/TRAIN_1_IRREGULAR_STEPS_V2.json
VALIDATION Dataset loaded. Size: 3974


Finally, you can create your pytorch loader and have fun with your networks !

In [5]:
from torch.utils.data import DataLoader

loader = DataLoader(train_dataset, batch_size=1, drop_last=True)
feature,is_maneuver,maneuver_dv,maneuver_time =next(iter(loader))
print(f"features shape (batch size * nb of meas * nb of feature):{feature.shape}\nis maneuver: {is_maneuver.item()}\ndv (m/s): {maneuver_dv.item()}\nmaneuver date (seconds from the observation start): {maneuver_time.item()}")

features shape (batch size * nb of meas * nb of feature):torch.Size([1, 1000, 3])
is maneuver: 0
dv (m/s): -0.2656478013
maneuver date (seconds from the observation start): 262984.8449901311


To load the test dataset, you should set the dataset type to TEST:

In [6]:
test_dataset_path="/kaggle/working/maneuver-detection/TEST_FILE_PUBLIC.json"
test_dataset= ManeuverDetectionDataset(test_dataset_path,dataset_type="TEST")
test_loader = DataLoader(test_dataset, batch_size=1, drop_last=True)
feature,is_maneuver,maneuver_dv,maneuver_time =next(iter(test_loader))
print(f"features shape (batch size * nb of meas * nb of feature):{feature.shape}\nis maneuver: {is_maneuver.item()}\ndv (m/s): {maneuver_dv.item()}\nmaneuver date (seconds from the observation start): {maneuver_time.item()}")




**********TEST DATASET *********
Validation/Train ratio: 0.1
Samples filtered? NO
Samples evenly spaced? False
path: /kaggle/working/maneuver-detection/TEST_FILE_PUBLIC.json
loading dataset. Ready in a minute!
TEST Dataset loaded. Size: 1559
features shape (batch size * nb of meas * nb of feature):torch.Size([1, 1000, 3])
is maneuver: 0
dv (m/s): 0.0
maneuver date (seconds from the observation start): 0.0


for evenly spaced dataset, you should use the option fixed_step : 


In [7]:
evenly_spaced_dataset_path="/kaggle/working/maneuver-detection/TRAIN_1_EVENLY_SPACED_V2.json"
evenly_spaced_dataset= ManeuverDetectionDataset(evenly_spaced_dataset_path,fixed_step=True)
evenly_spaced_loader = DataLoader(evenly_spaced_dataset, batch_size=1, drop_last=True)
feature,is_maneuver,maneuver_dv,maneuver_time =next(iter(evenly_spaced_loader))
print(f"features shape (batch size * nb of meas * nb of feature):{feature.shape}\nis maneuver: {is_maneuver.item()}\ndv (m/s): {maneuver_dv.item()}\nmaneuver date (seconds from the observation start): {maneuver_time.item()}")




**********TRAIN DATASET *********
Validation/Train ratio: 0.1
Samples filtered? NO
Samples evenly spaced? True
path: /kaggle/working/maneuver-detection/TRAIN_1_EVENLY_SPACED_V2.json
loading dataset. Ready in a minute!
TRAIN Dataset loaded. Size: 35762
features shape (batch size * nb of meas * nb of feature):torch.Size([1, 433, 3])
is maneuver: 0
dv (m/s): -0.2656478013
maneuver date (seconds from the observation start): 262984.8449901311


#  It is time to submit !

When you think that your score is great you can submit ! To get started, you can generate your prediction file :

In [8]:
from SubmissionGenerator import create_submission
import numpy as np
#fake prediction
pred=np.zeros((len(test_dataset),3))
pred[:,0]=np.ones((len(test_dataset))) # classification
pred[:,1]=0.01*np.ones((len(test_dataset))) #dv
pred[:,2]=24*3600*np.ones((len(test_dataset))) #time of the maneuver
create_submission(pred,"/kaggle/working/prediction")


Then, commit the file prediction.csv in the gitlab repository of your team. Don't forget to pull the repo before! Wait a few moment and you should see your score appear in the leaderboard : http://34.163.85.74:5006/dashboard